# [Exporting 🤗 Transformers Models](https://huggingface.co/docs/transformers/serialization)

## Using TorchScript in Python

Below is an example, showing how to save, load models as well as how to use the trace for inference. 

In [1]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import audio2numpy as a2n
import numpy as np
import torch
import torchaudio


In [2]:
def load_mp3_to_array_fn(path: str):
    r"""replace: torchaudio.load(path) which cannot load .mp3"""
    speech_array, _ = a2n.audio_from_file(path)
    speech_array = torch.tensor(np.array([speech_array]))
    resampler = torchaudio.transforms.Resample(48_000, 16_000)
    tensor = resampler(speech_array).squeeze().numpy()
    return tensor


### Saving a model

In [3]:
# Instantiating the model
model = Wav2Vec2ForCTC.from_pretrained(
    "airesearch/wav2vec2-large-xlsr-53-th").to("cuda")
model.eval()


Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

/home/somenoe/miniconda3/envs/py37/lib/python3.7/site-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureExtractor(
      (conv_layers): ModuleList(
        (0): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
        (1): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
        (2): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
        (3): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
        (4): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,)

In [4]:
# Instantiating the processor
processor = Wav2Vec2Processor.from_pretrained(
    "airesearch/wav2vec2-large-xlsr-53-th")


Downloading:   0%|          | 0.00/215 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/762 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/181 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
# Test original model from huggingface
path = r"./mp3/common_voice_th_23646618.mp3"

tensor = load_mp3_to_array_fn(path)

inputs = processor(tensor, sampling_rate=16_000,
                   return_tensors="pt", padding=True)

input_values = inputs.input_values.to("cuda")


/home/somenoe/miniconda3/envs/py37/lib/python3.7/site-packages/transformers/feature_extraction_utils.py:158: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1640811797118/work/torch/csrc/utils/tensor_new.cpp:201.)
  tensor = as_tensor(value)


In [7]:
# Creating the trace
traced_model = torch.jit.trace(model, input_values, strict=False)
torch.jit.save(traced_model, "./model/airesearch-wav2vec2-large-xlsr-53-th.pt")


### Loading a model

In [8]:
loaded_model = torch.jit.load(
    "./model/airesearch-wav2vec2-large-xlsr-53-th.pt")
loaded_model.eval()


RecursiveScriptModule(
  original_name=Wav2Vec2ForCTC
  (wav2vec2): RecursiveScriptModule(
    original_name=Wav2Vec2Model
    (feature_extractor): RecursiveScriptModule(
      original_name=Wav2Vec2FeatureExtractor
      (conv_layers): RecursiveScriptModule(
        original_name=ModuleList
        (0): RecursiveScriptModule(
          original_name=Wav2Vec2LayerNormConvLayer
          (conv): RecursiveScriptModule(original_name=Conv1d)
          (layer_norm): RecursiveScriptModule(original_name=LayerNorm)
        )
        (1): RecursiveScriptModule(
          original_name=Wav2Vec2LayerNormConvLayer
          (conv): RecursiveScriptModule(original_name=Conv1d)
          (layer_norm): RecursiveScriptModule(original_name=LayerNorm)
        )
        (2): RecursiveScriptModule(
          original_name=Wav2Vec2LayerNormConvLayer
          (conv): RecursiveScriptModule(original_name=Conv1d)
          (layer_norm): RecursiveScriptModule(original_name=LayerNorm)
        )
        (3): Recu

### Testing the model

In [9]:
with torch.no_grad():
    logits = model(input_values).logits

predicted_ids = torch.argmax(logits, dim=-1)

print("Prediction:", processor.batch_decode(predicted_ids))


Prediction: ['พวกเรา อาจจะ ต้องการ ความช่วยเหลือ จาก เธอ ในที่สุด']


In [10]:
with torch.no_grad():
    logits = loaded_model(input_values)['logits']

predicted_ids = torch.argmax(logits, dim=-1)

print("Prediction:", processor.batch_decode(predicted_ids))


Prediction: ['พวกเรา อาจจะ ต้องการ ความช่วยเหลือ จาก เธอ ในที่สุด']
